<a href="https://colab.research.google.com/github/SugarC21/colab_Open_WebUI/blob/main/Colab_Open_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open WebUI
All data will persist in Google Drive under 'open_WebUI' folder

In [ ]:
# Mount Google Drive and create folder structure
from google.colab import drive
drive.mount('/content/drive')

# Create main directory and subdirectories
!mkdir -p '/content/drive/MyDrive/open_WebUI/ollama_models'
!mkdir -p '/content/drive/MyDrive/open_WebUI/venv'
!mkdir -p '/content/drive/MyDrive/open_WebUI/config'

!sudo chmod -R 777 '/content/drive/MyDrive/open_WebUI/ollama_models'  # Full access
!ls -ld '/content/drive/MyDrive/open_WebUI/ollama_models'  # Verify permissions

## Dependencies
Install Ollama with persistent storage:

In [ ]:
!sudo apt-get update
!sudo apt-get install -y python3.11 python3.11-venv python3.11-dev lshw

# Create/update virtual environment in Drive
!python3.11 -m venv '/content/drive/MyDrive/open_WebUI/venv' --upgrade-deps


!curl -fsSL https://ollama.com/install.sh | sh

# Set custom model storage path
%env OLLAMA_MODELS=/content/drive/MyDrive/open_WebUI/ollama_models

## Install Open WebUI
Persistent installation in dedicated folder:

In [ ]:
# Install Open WebUI
!'/content/drive/MyDrive/open_WebUI/venv/bin/python' -m pip install --upgrade pip
!'/content/drive/MyDrive/open_WebUI/venv/bin/pip' install open-webui

## Server Configuration
Store startup script in Drive:

In [ ]:
with open('/content/drive/MyDrive/open_WebUI/start_servers.py', 'w') as f:
    f.write('''
import subprocess
import threading
import time

def start_ollama():
    subprocess.run(['ollama', 'serve'])

def start_open_webui():
    subprocess.run([
        '/content/drive/MyDrive/open_WebUI/venv/bin/open-webui',
        'serve',
        '--port', '8081'
    ])

threading.Thread(target=start_ollama, daemon=True).start()
time.sleep(5)
threading.Thread(target=start_open_webui, daemon=True).start()
while True: time.sleep(1)
''')

## Start Servers
Public URL generation:

In [ ]:
# Start servers and expose
#!'/content/drive/MyDrive/open_WebUI/venv/bin/python' '/content/drive/MyDrive/open_WebUI/start_servers.py' & sleep 20; echo | ssh -o StrictHostKeyChecking=no -p 443 -R0:localhost:8081 qr@a.pinggy.io

# Install required package
!pip install pyngrok -q

import subprocess
import time
from pyngrok import ngrok

# Set your ngrok auth token here
from google.colab import userdata
ngrok_auth_token = userdata.get('ngrok_auth_token')
ngrok_auth_token = ngrok_auth_token

# Start servers using subprocess
webui_process = subprocess.Popen([
    '/content/drive/MyDrive/open_WebUI/venv/bin/python',
    '/content/drive/MyDrive/open_WebUI/start_servers.py'
])

# Initialize ngrok with auth token
ngrok_process = subprocess.Popen(
    [
        'ngrok', 'http', '8081',
        '--authtoken', ngrok_auth_token,
        '--log', 'stdout'  # Direct logs to standard output
    ],
    stdout=subprocess.PIPE,  # Capture output
    stderr=subprocess.STDOUT,  # Merge stderr with stdout
    universal_newlines=True  # Handle text output
)

# Wait for tunnels to establish
time.sleep(25)

# Get public URL using ngrok API
try:
    tunnels = subprocess.check_output(['curl', '-s', 'http://localhost:4040/api/tunnels'])
    public_url = str(tunnels).split('public_url":"')[1].split('"')[0]
    print(f"\nOpen WebUI Public URL: {public_url.replace('http://', 'https://')}\n")
except Exception as e:
    print(f"Error getting ngrok URL: {str(e)}")

# Keep the cell alive
import threading
def print_ngrok_output():
    for line in iter(ngrok_process.stdout.readline, ''):
        print(line, end='')

threading.Thread(target=print_ngrok_output, daemon=True).start()

# Test write access
!touch '/content/drive/MyDrive/open_WebUI/ollama_models/permission_test'
!ls '/content/drive/MyDrive/open_WebUI/ollama_models'  # Should show test file
!rm '/content/drive/MyDrive/open_WebUI/ollama_models/permission_test'